<a href="https://colab.research.google.com/github/ronykroy/NLP/blob/master/H20_multiClass_newsGroups_Top4_noGPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# No GPU excluded deeplearning models

In [0]:
!apt-get install default-jre
!java -version
# install Java.. because.. there is nothing in this colab VM

Reading package lists... Done
Building dependency tree       
Reading state information... Done
default-jre is already the newest version (2:1.11-68ubuntu1~18.04.1).
default-jre set to manually installed.
0 upgraded, 0 newly installed, 0 to remove and 28 not upgraded.
openjdk version "11.0.4" 2019-07-16
OpenJDK Runtime Environment (build 11.0.4+11-post-Ubuntu-1ubuntu218.04.3)
OpenJDK 64-Bit Server VM (build 11.0.4+11-post-Ubuntu-1ubuntu218.04.3, mixed mode, sharing)


In [0]:
! pip install h2o
# H2O too to be installed SMH

     |████████████████████████████████| 123.7MB 88kB/s 
  Created wheel for h2o: filename=h2o-3.26.0.9-py2.py3-none-any.whl size=123719079 sha256=034e9de11698719783d75caa171186369ffe11221013b3beb73713db5fa5db04
  Stored in directory: /root/.cache/pip/wheels/ff/a9/36/487108894572fb92f9262ef16b3db700601992274b9705b214
Successfully built h2o


In [0]:
import h2o
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [0]:
h2o.init(max_mem_size="4G")

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.4" 2019-07-16; OpenJDK Runtime Environment (build 11.0.4+11-post-Ubuntu-1ubuntu218.04.3); OpenJDK 64-Bit Server VM (build 11.0.4+11-post-Ubuntu-1ubuntu218.04.3, mixed mode, sharing)
  Starting server from /usr/local/lib/python3.6/dist-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpmz02m0xy
  JVM stdout: /tmp/tmpmz02m0xy/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmpmz02m0xy/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O cluster uptime:,02 secs
H2O cluster timezone:,Etc/UTC
H2O data parsing timezone:,UTC
H2O cluster version:,3.26.0.9
H2O cluster version age:,6 days
H2O cluster name:,H2O_from_python_unknownUser_q26nfd
H2O cluster total nodes:,1
H2O cluster free memory:,4 Gb
H2O cluster total cores:,2
H2O cluster allowed cores:,2
H2O cluster status:,"accepting new members, healthy"


In [0]:
from sklearn.datasets import fetch_20newsgroups
dataset = fetch_20newsgroups(shuffle=True, random_state=1, remove=('headers', 'footers', 'quotes'))
documents = dataset.data

INFO:sklearn.datasets.twenty_newsgroups:Downloading 20news dataset. This may take a few minutes.
INFO:sklearn.datasets.twenty_newsgroups:Downloading dataset from https://ndownloader.figshare.com/files/5975967 (14 MB)


In [0]:
df = pd.DataFrame({'label':dataset.target, 'text':dataset.data})
df_trn, df_test = train_test_split(df, stratify = df['label'], test_size = 0.15, random_state = 11) 

In [0]:
reviews = h2o.H2OFrame(df)
reviews_train = h2o.H2OFrame(df_trn)
reviews_test = h2o.H2OFrame(df_test)

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [0]:
reviews_train.head(2)

label,text
0,"In emarsh@hernes-sun.Eng.Sun.COM (Eric Huh? Please explain. Is there a problem because I based my morality on something that COULD be wrong? Gosh, there's a heck of a lot of stuff that I believe that COULD be wrong, and that comes from sources that COULD be wrong. What do you base your belief on atheism on? Your knowledge and reasoning? COuldn't that be wrong? MAC -- **************************************************************** Michael A. Cobb ""...and I won't raise taxes on the middle University of Illinois class to pay for my programs."" Champaign-Urbana -Bill Clinton 3rd Debate cobb@alexia.lis.uiuc.edu"
6,"Nintendo 8 bit system, power pad, light gun (zapper), 2 controllers Games: Supermario, duck hunt, power field, and wings. Asking $80."


In [0]:
# Set Stop Words
# The STOP WORDS we are importing are from the nltk package
import pandas as pd
import os

# Use local data file or download from GitHub
docker_data_path = "/home/h2o/data/nlp/stopwords.csv"
if os.path.isfile(docker_data_path):
  data_path = docker_data_path
else:
  data_path = "https://raw.githubusercontent.com/h2oai/h2o-tutorials/master/h2o-world-2017/nlp/stopwords.csv"

STOP_WORDS = pd.read_csv(data_path, header=0)
STOP_WORDS = list(STOP_WORDS['STOP_WORD'])

In [0]:
# TODO: review how effective this is in removing junk \r \n etc
def tokenize(sentences, stop_word = STOP_WORDS):
    tokenized = sentences.tokenize("\\W+")
    tokenized_lower = tokenized.tolower()
    tokenized_filtered = tokenized_lower[(tokenized_lower.nchar() >= 2) | (tokenized_lower.isna()),:]
    tokenized_words = tokenized_filtered[tokenized_filtered.grep("[0-9]",invert=True,output_logical=True),:]
    tokenized_words = tokenized_words[(tokenized_words.isna()) | (~ tokenized_words.isin(STOP_WORDS)),:]
    return tokenized_words

In [0]:
%%time
# Break reviews into sequence of words
words_train = tokenize(reviews_train["text"]) # to check against leakage here...
words_test = tokenize(reviews_test["text"]) 

CPU times: user 6.83 s, sys: 140 ms, total: 6.97 s
Wall time: 23.6 s


In [0]:
words_train.head()

C1
exodus
eng
sun
com
emarsh
hernes
sun
eng
sun
com


In [0]:
%%time
# Train Word2Vec Model
from h2o.estimators.word2vec import H2OWord2vecEstimator

w2v_model = H2OWord2vecEstimator(vec_size = 200, model_id = "w2v.hex")
w2v_model.train(training_frame=words_train) # train only on  the train_words  we just got

word2vec Model Build progress: |██████████████████████████████████████████| 100%
CPU times: user 1.84 s, sys: 187 ms, total: 2.03 s
Wall time: 4min 39s


In [0]:
# Sanity check - find synonyms for the word 'president'
w2v_model.find_synonyms("president", count = 5) 

OrderedDict([('mubarak', 0.7638683915138245),
             ('yeltsin', 0.747818648815155),
             ('stephanopoulos', 0.7300563454627991),
             ('miyazawa', 0.7019920945167542),
             ('dole', 0.6614255309104919)])

In [0]:
# Calculate a vector for each review in each data set
review_vecs_train = w2v_model.transform(words_train, aggregate_method = "AVERAGE")
review_vecs_test = w2v_model.transform(words_test, aggregate_method = "AVERAGE")

In [0]:
review_vecs_train

C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,C15,C16,C17,C18,C19,C20,C21,C22,C23,C24,C25,C26,C27,C28,C29,C30,C31,C32,C33,C34,C35,C36,C37,C38,C39,C40,C41,C42,C43,C44,C45,C46,C47,C48,C49,C50,C51,C52,C53,C54,C55,C56,C57,C58,C59,C60,C61,C62,C63,C64,C65,C66,C67,C68,C69,C70,C71,C72,C73,C74,C75,C76,C77,C78,C79,C80,C81,C82,C83,C84,C85,C86,C87,C88,C89,C90,C91,C92,C93,C94,C95,C96,C97,C98,C99,C100,C101,C102,C103,C104,C105,C106,C107,C108,C109,C110,C111,C112,C113,C114,C115,C116,C117,C118,C119,C120,C121,C122,C123,C124,C125,C126,C127,C128,C129,C130,C131,C132,C133,C134,C135,C136,C137,C138,C139,C140,C141,C142,C143,C144,C145,C146,C147,C148,C149,C150,C151,C152,C153,C154,C155,C156,C157,C158,C159,C160,C161,C162,C163,C164,C165,C166,C167,C168,C169,C170,C171,C172,C173,C174,C175,C176,C177,C178,C179,C180,C181,C182,C183,C184,C185,C186,C187,C188,C189,C190,C191,C192,C193,C194,C195,C196,C197,C198,C199,C200
-0.0272315,0.0447182,0.00395584,-0.00103804,0.0161892,-0.0812052,0.133627,-0.0511577,-0.0889799,0.124263,-0.0453859,0.0762203,0.118014,-0.0685731,0.0200657,0.00333286,-0.0163878,0.0765017,0.0154177,0.0251768,-0.00729409,-0.0181911,0.0336502,0.047907,-0.0903314,-0.15147,0.000226911,-0.0614119,0.0239938,-0.0540347,0.0250888,0.0465415,-0.0356307,-0.0455423,-0.101829,0.0218434,0.0511463,-0.0762411,-0.0752921,0.069794,0.0275509,0.0741611,-0.067279,0.0570024,0.0515592,-0.0528615,0.0482564,-0.0355378,0.0301782,-0.0603858,0.00109951,0.140561,-0.0918544,-0.0632122,0.164667,0.135907,-0.0863429,0.0774204,0.105685,-0.0409307,-0.0462632,0.0389658,-0.0287536,0.00582462,-0.0721771,0.0746741,0.0123911,0.134425,-0.0152183,-0.0455991,0.0565641,-0.0527641,0.0991426,0.022165,-0.0336743,-0.0717936,0.0503688,-0.0602452,-0.06796,-0.0550818,-0.0623226,0.0889786,0.00203583,0.0957947,0.127679,0.010302,0.198284,-0.021792,-0.0911999,0.0279977,-0.103313,0.0586636,0.112513,0.014047,0.0616726,-0.0105398,0.0555735,0.0260148,0.0207946,-0.00799799,0.038423,-0.0310265,0.0636431,-0.0952039,0.0467242,0.068347,0.0649849,-0.034794,-0.0982516,0.00314061,0.127926,-0.00404314,0.0510209,0.199456,0.0740701,0.00275927,-0.111597,-0.0873825,-0.0567772,0.0433345,-0.0169983,-0.100543,-0.0399619,-0.00381621,-0.00923826,-0.0256027,-0.0589539,-0.064184,0.0888876,0.186126,-0.0741492,-0.127809,-0.0564379,-0.00374483,0.0338251,0.0459947,0.106609,-0.0224484,-0.00248351,-0.0562611,0.0064511,0.065706,0.0441077,-0.102793,-0.159998,-0.0893802,0.0308431,0.0277686,-0.0381355,0.100356,-0.0286807,-0.0705123,-0.0216143,-0.0725225,-0.0239273,0.0937567,-0.0628295,0.04349,0.0225817,-0.00681961,-0.0885676,-0.0892715,0.0253703,-0.109605,0.0839432,0.0330573,0.203927,0.0980184,0.0382779,-0.0333602,-0.00591795,-0.107682,-0.0848641,0.0466326,-0.0366818,0.021661,-0.0774358,0.0591447,-0.0550754,0.0125347,-0.119023,0.039204,0.063504,-0.0371309,0.0531926,-0.0274357,0.00724841,0.142316,-0.104463,-0.0149134,-0.0388674,-0.159046,0.000339531,-0.169874,-0.0795961,0.0393183,0.0320037,0.0653553,0.0579398,0.12861
0.0800445,0.0590833,0.0151346,-0.0733125,0.0955426,-0.117895,0.103903,0.108374,0.109424,-0.0164071,-0.0451256,0.00187026,-0.0223735,-0.0577826,0.0764738,-0.0146111,-0.0539079,0.0605723,-0.0149315,0.0670648,-0.0277649,0.0539545,0.00246655,0.0173048,-0.127877,-0.22268,0.135818,0.00133058,0.0713047,0.0175792,0.0055055,-0.072241,0.0936847,-0.0356662,-0.0525955,-0.114564,0.0838199,-0.00792547,-0.087195,0.0817973,-0.0268595,-0.0372004,0.0147953,0.0689144,0.0448704,0.0218123,-0.0301167,-0.0664948,-0.16284,-0.0647832,-0.00667036,0.201666,0.150103,-0.00324253,0.107105,0.00944501,0.0033579,0.206254,0.0355031,-0.0488562,-0.0077531,-0.0298783,-0.0915779,-0.00866011,-0.0675547,0.148874,-0.132939,0.17287,0.152048,-0.108804,-0.0181375,0.0523616,0.0226256,0.0567301,-0.0869163,0.0058073,0.0162543,-0.130478,-0.111549,-0.0181861,0.0082291,-0.0113235,-0.0217565,0.124349,0.126417,-0.143386,-0.126618,0.00674707,0.033905,-0.0456417,0.0755728,0.0854103,0.0809062,0.0186368,-0.0552429,-0.00569071,0.0426481,-0.0586837,0.0544269,-0.0308588,0.0756087,-0.13

In [0]:
# Add aggregated word embeddings 
ext_reviews_train = reviews_train.cbind(review_vecs_train)
ext_reviews_test = reviews_test.cbind(review_vecs_test)
# rename may be... all 'reviews' to 'news'
# ext becaus ewe are extending the reviews df with the vectors

In [0]:
ext_reviews_train["label_f"]  = ext_reviews_train["label"] .asfactor()


In [0]:
ext_reviews_test["label_f"]  = ext_reviews_test["label"] .asfactor() # need this too for the conf matrix

In [0]:
# Splitting to test train.. along with the review vecs is difficult without the 'Train' boolean column
# hence redoing it from that stage..
# so that we'll be able to do this following cell

In [0]:
#ext_train = ext_reviews[ext_reviews["Train"] == "True"]
#ext_test = ext_reviews[ext_reviews["Train"] == "False"] 
# these are not needed now.. since these were already done...

In [0]:
#review_vecs.names
# had a 100 cols
# may be more would give a better model ..?

In [0]:
reviews.shape[0]

12291

In [0]:
from h2o.automl import H2OAutoML

In [0]:
x = ext_reviews_train.columns

In [0]:
# x= [x for x in ext_train.columns if x not in ['label', 'Train']]

In [0]:
print(x)

['label', 'text', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'C11', 'C12', 'C13', 'C14', 'C15', 'C16', 'C17', 'C18', 'C19', 'C20', 'C21', 'C22', 'C23', 'C24', 'C25', 'C26', 'C27', 'C28', 'C29', 'C30', 'C31', 'C32', 'C33', 'C34', 'C35', 'C36', 'C37', 'C38', 'C39', 'C40', 'C41', 'C42', 'C43', 'C44', 'C45', 'C46', 'C47', 'C48', 'C49', 'C50', 'C51', 'C52', 'C53', 'C54', 'C55', 'C56', 'C57', 'C58', 'C59', 'C60', 'C61', 'C62', 'C63', 'C64', 'C65', 'C66', 'C67', 'C68', 'C69', 'C70', 'C71', 'C72', 'C73', 'C74', 'C75', 'C76', 'C77', 'C78', 'C79', 'C80', 'C81', 'C82', 'C83', 'C84', 'C85', 'C86', 'C87', 'C88', 'C89', 'C90', 'C91', 'C92', 'C93', 'C94', 'C95', 'C96', 'C97', 'C98', 'C99', 'C100', 'C101', 'C102', 'C103', 'C104', 'C105', 'C106', 'C107', 'C108', 'C109', 'C110', 'C111', 'C112', 'C113', 'C114', 'C115', 'C116', 'C117', 'C118', 'C119', 'C120', 'C121', 'C122', 'C123', 'C124', 'C125', 'C126', 'C127', 'C128', 'C129', 'C130', 'C131', 'C132', 'C133', 'C134', 'C135', 'C136', 'C

In [0]:
# Identify predictors and response
x = ext_reviews_train.columns
x = [x for x in ext_reviews_train.columns if x not in ['label', 'Train','text','label_f']] # remove columns that can leak data
y = "label_f"
# other_cols = ""
# x.remove(y)

# For binary classification, response should be a factor
#train[y] = train[y].asfactor()
#test[y] = test[y].asfactor()
# Already done

In [0]:
print(x)

['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'C11', 'C12', 'C13', 'C14', 'C15', 'C16', 'C17', 'C18', 'C19', 'C20', 'C21', 'C22', 'C23', 'C24', 'C25', 'C26', 'C27', 'C28', 'C29', 'C30', 'C31', 'C32', 'C33', 'C34', 'C35', 'C36', 'C37', 'C38', 'C39', 'C40', 'C41', 'C42', 'C43', 'C44', 'C45', 'C46', 'C47', 'C48', 'C49', 'C50', 'C51', 'C52', 'C53', 'C54', 'C55', 'C56', 'C57', 'C58', 'C59', 'C60', 'C61', 'C62', 'C63', 'C64', 'C65', 'C66', 'C67', 'C68', 'C69', 'C70', 'C71', 'C72', 'C73', 'C74', 'C75', 'C76', 'C77', 'C78', 'C79', 'C80', 'C81', 'C82', 'C83', 'C84', 'C85', 'C86', 'C87', 'C88', 'C89', 'C90', 'C91', 'C92', 'C93', 'C94', 'C95', 'C96', 'C97', 'C98', 'C99', 'C100', 'C101', 'C102', 'C103', 'C104', 'C105', 'C106', 'C107', 'C108', 'C109', 'C110', 'C111', 'C112', 'C113', 'C114', 'C115', 'C116', 'C117', 'C118', 'C119', 'C120', 'C121', 'C122', 'C123', 'C124', 'C125', 'C126', 'C127', 'C128', 'C129', 'C130', 'C131', 'C132', 'C133', 'C134', 'C135', 'C136', 'C137', 'C138', 'C1

In [0]:
%%time
# Run AutoML for 20 base models (limited to 1 hour max runtime by default)
aml = H2OAutoML(max_models=20, seed=1,exclude_algos=  ["DeepLearning"])#, max_runtime_secs =1200)
aml.train(x=x, y=y, training_frame=ext_reviews_train)

AutoML progress: |████████████████████████████████████████████████████████| 100%
CPU times: user 24.6 s, sys: 1.71 s, total: 26.3 s
Wall time: 1h 26min 28s


In [0]:
# View the AutoML Leaderboard
lb = aml.leaderboard
lb.head(rows=lb.nrows)  # Print all rows instead of default (10 rows)

model_id,mean_per_class_error,logloss,rmse,mse
StackedEnsemble_AllModels_AutoML_20191105_093933,0.362748,1.18802,0.604074,0.364905
StackedEnsemble_BestOfFamily_AutoML_20191105_093933,0.368098,1.24817,0.62167,0.386473
GBM_1_AutoML_20191105_093933,0.374453,1.22667,0.585595,0.342921
GBM_2_AutoML_20191105_093933,0.377452,1.22584,0.600862,0.361035
DRF_1_AutoML_20191105_093933,0.382185,1.86671,0.685978,0.470566
GBM_3_AutoML_20191105_093933,0.44718,1.75194,0.758814,0.575799
GLM_grid_1_AutoML_20191105_093933_model_1,0.527445,2.61496,0.922908,0.851759


In [0]:
#aml.leader.confusion_matrix()
aml.leader.cross_validation_metrics_summary

Model Details
H2OStackedEnsembleEstimator :  Stacked Ensemble
Model Key:  StackedEnsemble_AllModels_AutoML_20191105_093933

No model summary for this model

ModelMetricsMultinomialGLM: stackedensemble
** Reported on train data. **

MSE: 0.028458619049308367
RMSE: 0.16869682584242174

ModelMetricsMultinomialGLM: stackedensemble
** Reported on cross-validation data. **

MSE: 0.3649052337007442
RMSE: 0.6040738644410502


<bound method ModelBase.cross_validation_metrics_summary of >

In [0]:
#ext_reviews_test.columns

In [0]:
aml.leader.confusion_matrix(ext_reviews_test) # 


Confusion Matrix: Row labels: Actual class; Column labels: Predicted class


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,Error,Rate
0,37.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,3.0,1.0,0.0,0.0,0.0,2.0,1.0,9.0,1.0,4.0,4.0,6.0,0.486111,35 / 72
1,0.0,51.0,8.0,3.0,3.0,3.0,1.0,4.0,2.0,0.0,0.0,0.0,1.0,1.0,9.0,0.0,0.0,1.0,0.0,1.0,0.420455,37 / 88
2,0.0,1.0,54.0,9.0,4.0,7.0,3.0,6.0,0.0,0.0,0.0,1.0,2.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.393258,35 / 89
3,0.0,2.0,9.0,48.0,15.0,0.0,4.0,2.0,0.0,0.0,0.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.454545,40 / 88
4,0.0,4.0,3.0,8.0,52.0,4.0,5.0,4.0,0.0,0.0,0.0,2.0,4.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.402299,35 / 87
5,0.0,6.0,4.0,0.0,0.0,75.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.157303,14 / 89
6,0.0,1.0,1.0,4.0,3.0,0.0,62.0,2.0,1.0,2.0,0.0,2.0,7.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.295455,26 / 88
7,0.0,2.0,0.0,0.0,2.0,0.0,0.0,59.0,11.0,0.0,0.0,0.0,5.0,1.0,1.0,0.0,5.0,0.0,1.0,2.0,0.337079,30 / 89
8,0.0,1.0,0.0,0.0,2.0,0.0,5.0,12.0,59.0,0.0,0.0,0.0,4.0,1.0,2.0,0.0,2.0,0.0,1.0,1.0,0.344444,31 / 90
9,0.0,2.0,0.0,0.0,1.0,0.0,0.0,4.0,3.0,73.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.179775,16 / 89



See the whole table with table.as_data_frame()


In [0]:
con_mat = aml.leader.confusion_matrix(ext_reviews_test).as_data_frame()

In [0]:
con_mat.iloc[0:20,0:20]

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,37.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,3.0,1.0,0.0,0.0,0.0,2.0,1.0,9.0,1.0,4.0,4.0,6.0
1,0.0,51.0,8.0,3.0,3.0,3.0,1.0,4.0,2.0,0.0,0.0,0.0,1.0,1.0,9.0,0.0,0.0,1.0,0.0,1.0
2,0.0,1.0,54.0,9.0,4.0,7.0,3.0,6.0,0.0,0.0,0.0,1.0,2.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0
3,0.0,2.0,9.0,48.0,15.0,0.0,4.0,2.0,0.0,0.0,0.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,4.0,3.0,8.0,52.0,4.0,5.0,4.0,0.0,0.0,0.0,2.0,4.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,6.0,4.0,0.0,0.0,75.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
6,0.0,1.0,1.0,4.0,3.0,0.0,62.0,2.0,1.0,2.0,0.0,2.0,7.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0
7,0.0,2.0,0.0,0.0,2.0,0.0,0.0,59.0,11.0,0.0,0.0,0.0,5.0,1.0,1.0,0.0,5.0,0.0,1.0,2.0
8,0.0,1.0,0.0,0.0,2.0,0.0,5.0,12.0,59.0,0.0,0.0,0.0,4.0,1.0,2.0,0.0,2.0,0.0,1.0,1.0
9,0.0,2.0,0.0,0.0,1.0,0.0,0.0,4.0,3.0,73.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0


In [0]:
np.sum(np.diag(con_mat.iloc[0:20,0:20]))/(con_mat.iloc[0:20,0:20].values.sum()) # sum of diagnoal elements/ sum of all in the matrix # 38733477 & SO 53762743

0.6560659599528857

### before making w2v exlusively from train data set: 0.6607298001737619
### after making w2v exlusively from train data set: 0.6395759717314488
### for 200 w2v model: 0.6584216725559482

In [0]:
predictions = aml.leader.predict(ext_reviews_test)

stackedensemble prediction progress: |████████████████████████████████████| 100%


In [0]:
predictions[:,:3]

predict,p0,p1
3,0.000600703,0.0157436
14,0.00127965,0.00714145
11,0.000263199,0.00020103
9,0.0246235,0.0383073
9,0.000118379,0.000172389
2,0.00724575,0.070791
7,0.00247004,0.00278652
3,0.00174441,0.0146932
9,0.0101546,0.0123529
7,0.0440112,0.0487843


In [0]:
predictions.as_data_frame().iloc[:,1:21] # all rows 1to 21 cols excluding last
 # predict colum gives the 1 hot notation of the answer

,p0,p1,p2,p3,p4,p5,p6,p7,p8,p9,p10,p11,p12,p13,p14,p15,p16,p17,p18,p19
0,0.000601,0.015744,0.040127,0.881029,0.035604,0.001165,0.010860,0.000622,0.000656,0.000625,0.000555,0.000627,0.007654,0.000598,0.000642,0.000578,0.000584,0.000581,0.000585,0.000565
1,0.001280,0.007141,0.001010,0.000918,0.001009,0.000933,0.001612,0.001657,0.001332,0.001124,0.000936,0.001261,0.001651,0.006126,0.963904,0.001219,0.001524,0.001587,0.001299,0.002476
2,0.000263,0.000201,0.000184,0.000163,0.000142,0.000234,0.000150,0.000144,0.000167,0.000149,0.000128,0.991691,0.002008,0.000238,0.000455,0.000162,0.002496,0.000485,0.000398,0.000143
3,0.024623,0.038307,0.016645,0.014828,0.016172,0.013226,0.029931,0.018254,0.020971,0.404877,0.179898,0.026095,0.024374,0.026821,0.034502,0.023782,0.019484,0.017757,0.023432,0.026019
4,0.000118,0.000172,0.000108,0.000101,0.000112,0.000102,0.000402,0.000198,0.000188,0.988880,0.008305,0.000108,0.000120,0.000105,0.000148,0.000117,0.000305,0.000124,0.000169,0.000118
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1693,0.002872,0.001866,0.001850,0.001794,0.002732,0.001590,0.003930,0.078471,0.858530,0.004564,0.003370,0.002904,0.002197,0.006467,0.010861,0.002170,0.005056,0.003568,0.003336,0.001872
1694,0.044011,0.048784,0.051836,0.054011,0.059298,0.027214,0.049512,0.072580,0.063831,0.064726,0.040737,0.040769,0.065613,0.036203,0.051162,0.037457,0.050568,0.041107,0.049380,0.051199
1695,0.000860,0.000603,0.000602,0.000567,0.000801,0.000577,0.001584,0.145535,0.834886,0.001759,0.001354,0.000905,0.000857,0.001798,0.002935,0.000609,0.001730,0.000838,0.000623,0.000575
1696,0.002788,0.035638,0.067362,0.442744,0.350236,0.004862,0.016837,0.004339,0.004925,0.002916,0.002766,0.002961,0.042047,0.002832,0.003871,0.002554,0.002642,0.002583,0.002631,0.002466


In [0]:
np.argsort(-predictions.as_data_frame().iloc[:,1:21].values, axis=1)[:, :4] # SO 54923349

array([[ 3,  2,  4,  1],
       [14,  1, 13, 19],
       [11, 16, 12, 17],
       ...,
       [ 8,  7, 14, 13],
       [ 3,  4,  2, 12],
       [ 8,  7,  6, 14]])

In [0]:
preds_top4 = pd.DataFrame(np.argsort(-predictions.as_data_frame().iloc[:,1:21].values, axis=1)[:, :4] )

In [0]:
target_test = ext_reviews_test['label'].as_data_frame()

In [0]:
np.sum(target_test.label == preds_top4[0])/preds_top4.shape[0] # yup matches what we got a few cells higher up

0.6560659599528857

In [0]:
match1 = np.sum(target_test.label == preds_top4[0])
match2 = np.sum(target_test.label == preds_top4[1])
match3 = np.sum(target_test.label == preds_top4[2])
match4 = np.sum(target_test.label == preds_top4[3])

In [0]:
# top 3
(match1+match2+match3)/preds_top4.shape[0]

0.842756183745583

In [0]:
# Top 4
(match1+match2+match3+match4)/preds_top4.shape[0]

0.875736160188457